In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin

In [25]:
class NullHandler_cc(BaseEstimator, TransformerMixin):
    def __init__(self, num_fill='mean', cat_fill= 'most_frequent'):
        self.num_fill= num_fill
        self.cat_fill= cat_fill
        
        
    def fit(self,df,num_cols=['a2', 'a3', 'a8', 'a11', 'a14', 'a15']):
        if type(df) != pd.DataFrame:
            raise TypeError("Pandas DataFrame Expected")
        
        self.df= df
        self.num_cols= num_cols
        df.columns= list(map(str.lower, list(df.columns)))
        self.cat_cols= list( set(list(df.columns)) - set(self.num_cols))
        
        self.cat_fill_vals={}
        for col in self.cat_cols:
            self.cat_fill_vals[col]= df[col].value_counts().idxmax()
        
        df[self.num_cols]= df[self.num_cols].replace('?', np.NAN)
        df['a2']= df['a2'].astype('float64')
        df['a11']= df['a11'].astype('float64')
        df['a14']= df['a14'].astype('float64')
        df['a15']= df['a15'].astype('float64')
        self.num_fill_vals= dict(df.mean())
        
        return self
    
    def transform(self, df):
        df[self.num_cols]= df[self.num_cols].replace('?', np.NAN)
        df['a2']= df['a2'].astype('float64')
        df['a11']= df['a11'].astype('float64')
        df['a14']= df['a14'].astype('float64')
        df['a15']= df['a15'].astype('float64')
        self.cat_fill_vals.update(self.num_fill_vals)
        fill_vals= self.cat_fill_vals
        df_ = df.fillna(value= fill_vals)
        
        return df_
        
        

In [4]:
def handling_categorical_attribute(df, attributes=[]):
    df[['a1', 'a9', 'a10', 'a12']]= df[['a1', 'a9', 'a10', 'a12']].apply(le.fit_transform)
    
    multivalcatcolumns= ['a4', 'a5', 'a6', 'a7', 'a13']
    df_list=[df]
    for col in multivalcatcolumns:
        ohe= OneHotEncoder(handle_unknown= 'ignore', sparse=False)
        temp= ohe.fit_transform(df[[col]])
        df_temp= pd.DataFrame(temp, columns=[col+x for x in ohe.categories_])
        df_list.append(df_temp)
        del ohe

    df_temp= pd.concat(df_list, axis=1)
    df_temp.drop(multivalcatcolumns, axis=1, inplace= True)
    
    return df_temp

In [5]:
def scaling_data(df):
    scaler= StandardScaler()
    return scaler.fit_transform(df)

In [4]:
X_train= pd.read_csv("/home/souvik/ML_projects/P1_Credit_card_Approval/data/X_train.csv")
y_train= pd.read_csv("/home/souvik/ML_projects/P1_Credit_card_Approval/data/y_train.csv")
X_train.head()

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15
0,b,20.17,8.170,u,g,aa,v,1.960,t,t,14,f,g,00060,158
1,a,27.58,3.000,u,g,m,v,2.790,f,t,1,t,g,00280,10
2,b,39.58,13.915,u,g,w,v,8.625,t,t,6,t,g,00070,0
3,b,41.17,1.250,y,p,w,v,0.250,f,f,0,f,g,00000,195
4,b,22.25,0.460,u,g,k,v,0.125,f,f,0,t,g,00280,55


In [26]:
#print(X_train.isna().sum())

null_handler= NullHandler_cc()
null_handler.fit(X_train)
X_train_no_null= null_handler.transform(X_train)
print(X_train_no_null.isna().sum())

a1     0
a2     0
a3     0
a4     0
a5     0
a6     0
a7     0
a8     0
a9     0
a10    0
a11    0
a12    0
a13    0
a14    0
a15    0
dtype: int64


In [7]:
X_train.apply(str.count("?").sum, axis=0)

TypeError: count() takes at least 1 argument (0 given)

In [13]:
X_train.a1.astype('str').count('?')

KeyError: 'Requested level (?) does not match index name (None)'

In [8]:
X_train['a1'].value_counts()

b    378
a    164
?     10
Name: a1, dtype: int64

In [9]:
X_train['a4'].value_counts()

u    412
y    132
?      6
l      2
Name: a4, dtype: int64

In [10]:
X_train['a5'].value_counts()

g     412
p     132
?       6
gg      2
Name: a5, dtype: int64